In [4]:
import pickle
import numpy as np
import math
import time
%time

# Fix error with TF and Keras
import tensorflow as tf
from tensorflow import python
tf.python = tf

print('Modules loaded.')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs
Modules loaded.


In [5]:
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()

Using TensorFlow backend.


170500096/170498071 [==============================] - 56s 0us/step


In [7]:
# Shuffle the data
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

In [8]:
# Normalize the data features (rbg pixel values)
def normalize_grayscale(image_data):
    a = -0.5
    b = 0.5
    grayscale_min = 0
    grayscale_max = 255
    return a + ( ( (image_data - grayscale_min)*(b - a) )/( grayscale_max - grayscale_min ) )

#X_normalized = normalize_grayscale(X_train)
X_normalized = X_train / 255 - 0.5

In [9]:
# One Hot encode the labels 
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)

In [24]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import deserialize as layer_from_config
from keras.regularizers import l2, l1

In [26]:
model = Sequential()

#layer 1 - conv2d -> ReLU -> maxPool
model.add(Convolution2D(32, 5, 5, input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

#layer 2 - conv2d -> ReLU -> maxPool
model.add(Convolution2D(64, 5, 5))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

#layer 3 - fully connected -> relu -> dropout
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))

#layer 4 - fully connected -> relu -> dropout
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

#layer 5 - fully connexted -> softmax
# TODO: L2 regularization applied to output correctly??
model.add(Dense(10, activity_regularizer=l2(1e-6)))
model.add(Activation('softmax'))

/Users/alvanchow/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (5, 5), input_shape=(32, 32, 3...)`
  after removing the cwd from sys.path.
/Users/alvanchow/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5))`
  if __name__ == '__main__':


In [27]:
# Compile and train the model
# ~20-30 seconds per epoch on GPU
model.compile('adam', 'categorical_crossentropy', ['accuracy'])
%time history_10 = model.fit(X_normalized, y_one_hot, nb_epoch=10, validation_split=0.2, verbose=0)

/Users/alvanchow/anaconda/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


KeyboardInterrupt: 